# Preprocesamiento de datos (Tweets)

En este notebook se realiza el preprocesamiento de los datos que contienen los tweets para el entrenamiento de los modelos de aprendizaje automático extraidos de HuggingFace. Los datos han dido extraídos de la plataforma de Kaggle, en el siguiente [enlace](https://www.kaggle.com/datasets/alishafaghi/hashtag-tesla-tweets). El dataset contiene más de 150.000 tweets etiquetados con #tesla.

El objetivo es obtener un dataset con una única columna de tweets en inglés, para su posterior etiquetado por parte del alumno. El etiquetado de datos será discutido en la memoria, y realizado en los scripts de la carpeta `labelling`.

In [1]:
import pandas as pd
import yfinance as yf
from langdetect import detect
import preprocessor as p
import string
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\34644\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocesamiento del dataset de tweets

### Overview

En esta sección se realiza la carga de datos y análisis de alto nivel del datafrfame

In [2]:
df_tweets = pd.read_csv('../../data/raw/tesla_tweets.csv')
df_tweets.sample(5)

,Date & Time,Profile Picture Link,Twitter ID,Tweet Text,Tweet Link
105733,"September 08, 2022 at 04:19PM",http://pbs.twimg.com/profile_images/146513470...,@projekt100x,Wenn #Tesla im nächsten Jahr Ferraris operativ...,https://twitter.com/projekt100x/status/1567842...
78738,"August 01, 2022 at 01:17PM",http://pbs.twimg.com/profile_images/125846004...,@NatEco12,KIA EV6 GT DRIVEN! Has the Tesla Model 3 Perfo...,https://twitter.com/NatEco12/status/1554025914...
95051,"August 24, 2022 at 11:15AM",http://pbs.twimg.com/profile_images/151998749...,@cryptonoticing,RT @prudentwriters: Why #QuantumComputing Is E...,https://twitter.com/cryptonoticing/status/1562...
128464,"October 10, 2022 at 11:17AM",http://pbs.twimg.com/profile_images/965494239...,@kirillklip,RT @kirillklip: TNR Gold NSR Royalty Holding: ...,https://twitter.com/kirillklip/status/15793779...
139325,"October 25, 2022 at 04:08PM",http://pbs.twimg.com/profile_images/965494239...,@kirillklip,TNR Gold NSR Royalty Holding: President Albert...,https://twitter.com/kirillklip/status/15848871...


Como lo único que nos interesa es la columna de texto y su id, se descartan las demás columnas y se establece el tweet de cada post como índice del dataframe.

In [3]:

selected_columns = ['Tweet Link','Date & Time','Tweet Text']
df_tweets = df_tweets[selected_columns]
df_tweets.rename(columns={'Tweet Text':'Text', 'Tweet Link' : 'Link', 'Date & Time' : 'Date'}, inplace=True)
df_tweets.sample(5)

,Link,Date,Text
106507,https://twitter.com/BobWhit40143681/status/156...,"September 09, 2022 at 06:19PM",#Tesla Is Planning a New Business
43186,https://twitter.com/flybrandenburg/status/1536...,"June 12, 2022 at 11:04PM","New #GigaBerlin drone video from today, June 1..."
33040,https://twitter.com/TMukeshian/status/15309078...,"May 29, 2022 at 06:14PM",RT @globaltimesnews: #Tesla has adopted a doub...
28410,https://twitter.com/snband4/status/15285687233...,"May 23, 2022 at 07:19AM",RT @StrictlyChristo: Insurance companies are c...
25264,https://twitter.com/DaMiIkMan/status/152698119...,"May 18, 2022 at 10:11PM",RT @invest_answers: #Tesla gets kicked out of ...


In [4]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Link    152000 non-null  object
 1   Date    152000 non-null  object
 2   Text    152000 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


Comprobamos que la columna de fecha no está en formato datetime, por lo que se convierte a dicho formato. 

In [5]:
df_tweets['Date'] = pd.to_datetime(df_tweets['Date'], format='mixed') # dejamos que pandas infiera el formato de fecha con el formato mixed
print(df_tweets.info())
df_tweets.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Link    152000 non-null  object        
 1   Date    152000 non-null  datetime64[ns]
 2   Text    152000 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.5+ MB
None


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,"RT @invest_answers: Crypto news, #Bitcoin Whal..."
1,https://twitter.com/JotaGe2014/status/15131737...,2022-04-10 19:45:00,#Tesla tiene récord de autos vendidos. Es impr...
2,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
3,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
4,https://twitter.com/agseh/status/1513173864829...,2022-04-10 19:45:00,RT @RupiReportero_: 🙆‍♂️🚘 Al que le robaron la...


### Limpieza de datos

Se comprueba si existen valores nulos o atípicos, además de comprobar si existen tweets duplicados.


In [6]:
df_tweets.isnull().sum()

Link    0
Date    0
Text    0
dtype: int64

Al ser la fecha de publicación del tweet y el propio tweet en cuestion variables de tipos distintos, se hace su análisis por separado.

En la descripción del dataset en la página de Kaggle se indica que los tweets han sido extraídos entre Abril de 2022 y Noviembre de 2022. Por lo que se comprueba que no existen tweets con fecha anterior al 1 de enero de 2020. Comprobamos que no existen tweets con fechas posteriores ni anteriorers a las indicadas.

In [7]:
# Análisis de valores atípicos en la columna de fecha

(df_tweets['Date'].min(),
df_tweets['Date'].max())

(Timestamp('2022-04-10 19:44:00'), Timestamp('2022-11-12 15:11:00'))

### Preparación de la columna de text

En esta sección se realiza la limpieza de la columna de texto.Se eliminan los caracteres especiales, las menciones, los enlaces y los emojis utilizando la librería [tweet preprocessor](https://pypi.org/project/tweet-preprocessor/). Se convierte todo el texto a minúsculas, y seleccionamos sólo los tweets en inglés.	

Sería interesante poder elegir otros idiomas aparte del inglés para el etiquetado de datos, pero por simplicidad se ha decidido utilizar sólo el inglés, ya que es el idioma más utilizado en la plataforma de Twitter. Sería posible utilizar modelos preentrenados para la clasificación de sentimientos en diferentes idiomas, pero se ha decidido utilizar sólo el inglés para simplificar el proceso de etiquetado.

In [8]:
#dataframe vacío para almacenar los tweets que son en inglés
i = 0
df_aux = pd.DataFrame(columns=['Link','Date','Text'])
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)
start_time = time.time()

for index, row in df_tweets.iterrows():
    cleaned_text = p.clean(row['Text'])  # Limpia el texto y guarda el resultado

    # Esta celda tarda mucho, por lo que si se desea saber el progreso se puede imprimir el porcentaje de tweets procesados
    if i % (len(df_tweets.index)/10) == 0:
        print('Progreso:',i,'/',len(df_tweets.index),'=',i/len(df_tweets.index)*100,'%')
        print('Tiempo transcurrido:',time.time()-start_time,'segundos')
    i+=1
    
    try:
        if detect(cleaned_text) == 'en': # Si el texto está en inglés, lo guardamos en el dataframe auxiliar
            df_aux.loc[len(df_aux.index)] = [row['Link'], row['Date'], cleaned_text]
    except:
        pass


Progreso: 0 / 152000 = 0.0 %
Tiempo transcurrido: 0.2866506576538086 segundos
Progreso: 15200 / 152000 = 10.0 %
Tiempo transcurrido: 122.48489880561829 segundos
Progreso: 30400 / 152000 = 20.0 %
Tiempo transcurrido: 280.26022005081177 segundos
Progreso: 45600 / 152000 = 30.0 %
Tiempo transcurrido: 479.6051986217499 segundos
Progreso: 60800 / 152000 = 40.0 %
Tiempo transcurrido: 666.2523875236511 segundos
Progreso: 76000 / 152000 = 50.0 %
Tiempo transcurrido: 880.3665461540222 segundos
Progreso: 91200 / 152000 = 60.0 %
Tiempo transcurrido: 1093.8572807312012 segundos
Progreso: 106400 / 152000 = 70.0 %
Tiempo transcurrido: 1382.5346384048462 segundos
Progreso: 121600 / 152000 = 80.0 %
Tiempo transcurrido: 1629.175193786621 segundos
Progreso: 136800 / 152000 = 90.0 %
Tiempo transcurrido: 1904.1405940055847 segundos


In [9]:
def eliminar_puntuacion(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def eliminar_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

stop_words = set(stopwords.words('english'))

df_aux['Text'] = df_aux['Text'].apply(eliminar_puntuacion)
df_aux['Text'] = df_aux['Text'].apply(eliminar_stopwords)

print(len(df_aux.index))
df_aux.head(5)

124712


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes w...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Largest...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,chargingstations area I dont Tesla Powerwall a...


Como también fue notado en la caracterización de los datos, existen días que los mercados no están abiertos y por tanto no hay observaciones de las acciones. La estrategia de entrenamiento de los modelos de aprendizaje automático será la de predecir el precio de las acciones al día siguiente, por lo que se decide eliminar los tweets de los días en los que no hay observaciones de las acciones.

In [10]:
df_stocks = pd.read_csv('../../data/raw/tesla_stocks.csv')
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='%Y-%m-%d')

df_copy = df_aux.copy()


mask = df_copy['Date'].isin(df_stocks['Date'])
df_copy = df_copy[mask]

print(len(df_copy.index))


9


In [12]:
df_aux.to_csv('../../data/clean/', index=False)

PermissionError: [Errno 13] Permission denied: '../../data/clean/'